In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tuberculosis-drug-resistance-prediction-rif/X_testData_1.csv
/kaggle/input/tuberculosis-drug-resistance-prediction-rif/X_trainData_1.csv
/kaggle/input/tuberculosis-drug-resistance-prediction-rif/Y_trainData_1.csv
/kaggle/input/tuberculosis-drug-resistance-prediction-rif/Y_testData_1_nolabels_RIF.csv


In [2]:
X_1 = pd.read_csv("/kaggle/input/tuberculosis-drug-resistance-prediction-rif/X_trainData_1.csv")
Y_1 = pd.read_csv("/kaggle/input/tuberculosis-drug-resistance-prediction-rif/Y_trainData_1.csv")

In [3]:
X_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3393 entries, 0 to 3392
Columns: 222 entries, DEL_CF_410280_d918T_307_iniB to rrl-rrs_NC_snp
dtypes: int64(222)
memory usage: 5.7 MB


In [4]:
Y_inh = Y_1['RIF']
x = np.size(Y_inh)

In [5]:
for i in range(x):
    if(Y_inh[i] == -1):
        Y_inh.drop(i, inplace = True)
        X_1.drop(i,  inplace = True)

In [6]:
X = X_1.to_numpy()
Y = Y_inh.to_numpy()

In [7]:
from tensorflow import keras
from keras.layers import Dense, Dropout, Input, BatchNormalization
from keras.models import Model
from keras import regularizers
from keras.layers import concatenate
from keras.optimizers import Adam

In [8]:
#WDNN Implementation

input_  = keras.layers.Input(shape= (222,))
hidden1 = keras.layers.Dense(256, activation="relu")(input_)
hidden1 = BatchNormalization()(hidden1)
hidden1 = Dropout(0.5)(hidden1)
hidden2 = keras.layers.Dense(256, activation="relu")(hidden1)
hidden2 = BatchNormalization()(hidden2)
hidden2 = Dropout(0.5)(hidden2)
hidden2 = keras.layers.Dense(256, activation="relu")(hidden2)
hidden2 = BatchNormalization()(hidden2)
hidden2 = Dropout(0.5)(hidden2)
concat = keras.layers.concatenate([input_, hidden2])
preds = Dense(1, activation='sigmoid', kernel_regularizer=regularizers.l2(1e-8))(concat)
model = keras.models.Model(inputs = [input_], outputs = [preds])
opt = Adam(lr=np.exp(-1.0*9))
model.compile(optimizer=opt,loss= keras.losses.BinaryCrossentropy(),metrics= keras.metrics.AUC())
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 222)]        0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 256)          57088       input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 256)          1024        dense[0][0]                      
__________________________________________________________________________________________________
dropout (Dropout)               (None, 256)          0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [9]:
history = model.fit(X,Y,epochs = 200,validation_data = None)

Epoch 1/200
105/105 [==============================] - 0s 3ms/step - loss: 0.5158 - auc: 0.8493
Epoch 2/200
105/105 [==============================] - 0s 3ms/step - loss: 0.3457 - auc: 0.9346
Epoch 3/200
105/105 [==============================] - 0s 3ms/step - loss: 0.2917 - auc: 0.9481
Epoch 4/200
105/105 [==============================] - 0s 3ms/step - loss: 0.2521 - auc: 0.9604
Epoch 5/200
105/105 [==============================] - 0s 3ms/step - loss: 0.2311 - auc: 0.9656
Epoch 6/200
105/105 [==============================] - 0s 3ms/step - loss: 0.2241 - auc: 0.9678
Epoch 7/200
105/105 [==============================] - 0s 3ms/step - loss: 0.1990 - auc: 0.9728
Epoch 8/200
105/105 [==============================] - 0s 3ms/step - loss: 0.1799 - auc: 0.9781
Epoch 9/200
105/105 [==============================] - 0s 3ms/step - loss: 0.1704 - auc: 0.9813
Epoch 10/200
105/105 [==============================] - 0s 3ms/step - loss: 0.1548 - auc: 0.9836
Epoch 11/200
105/105 [=================

In [10]:
X_t = pd.read_csv("/kaggle/input/tuberculosis-drug-resistance-prediction-rif/X_testData_1.csv")
pred = model.predict(X_t.drop(columns = 'ID'))

In [11]:
k = pred

In [12]:
k = k.reshape((1000,))

In [13]:
Y_n = pd.read_csv("/kaggle/input/tuberculosis-drug-resistance-prediction-rif/Y_testData_1_nolabels_RIF.csv")

In [14]:
INH = pd.DataFrame({'ID':X_t['ID'],'RIF':k})
INH = pd.merge(Y_n,INH, on = 'ID')
INH.drop(columns = 'RIF_x',inplace = True)
INH.rename(columns = {'RIF_y':'RIF'}, inplace = True)
INH.to_csv('rif_res2.csv', index = False)

In [15]:
INH.head()

,ID,RIF
0,1,3.176054e-08
1,2,3.961452e-06
2,3,6.356316e-08
3,4,2.390986e-06
4,5,5.026424e-06


In [16]:
INH

,ID,RIF
0,1,3.176054e-08
1,2,3.961452e-06
2,3,6.356316e-08
3,4,2.390986e-06
4,5,5.026424e-06
...,...,...
937,996,9.999508e-01
938,997,9.967865e-01
939,998,1.000000e+00
940,999,9.999947e-01
